In [17]:
%load_ext autoreload
%autoreload 2

In [3]:
import anndata

adata = anndata.read_h5ad("../../data/raw/bowel_disease.h5ad")

In [4]:
from mmcontext.models.geneformer_model import GeneformerTranscriptomeProcessor

processor = GeneformerTranscriptomeProcessor(emb_label={"cluster_label": "cluster_label"}, nproc=16)

In [6]:
from datasets import Dataset

dataset = processor(adata)
hf_dataset = Dataset.from_dict(dataset)
Dataset.save_to_disk(hf_dataset, "../../../data/geneformer_datasets/bowel_disease2.dataset")

/Users/mengerj/repos/mmcontext/src/mmcontext/models/geneformer_model.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if "." in ensembl_ids[0]:


In [20]:
from geneformer import EmbExtractor

# Or try again with the code from geneformer
extractor = EmbExtractor(
    model_type="Pretrained",
    num_classes=0,
    emb_mode="cls",
    cell_emb_style="mean_pool",
    gene_emb_style="mean_pool",
    filter_data=None,
    max_ncells=1000,
    emb_layer=-1,
    emb_label=None,
    labels_to_plot=None,
    forward_batch_size=100,
    nproc=4,
    summary_stat=None,
    token_dictionary_file=None,
)

In [21]:
embs = extractor.extract_embs(
    "/Users/mengerj/repos/Geneformer/gf-12L-95M-i4096",
    "/Users/mengerj/repos/mmcontext/data/geneformer_datasets/bowel_disease.dataset",
    "../../../outputs/",
    output_prefix="bowel_disease2",
    output_torch_embs=True,
    cell_state=None,
)

  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: Placeholder storage has not been allocated on MPS device!